In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os


import pickle
from etcpy import etc
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

import requests

In [ ]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params.csv'),index_col=0)
params

In [ ]:
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)

In [ ]:
params2 = pd.read_csv(os.path.join(path,'data/model_enzyme_params.csv'),index_col=0)
params2=params2['dCpt_std'] = 1000
params2

In [2]:
mae = cobra.io.load_matlab_model('../models/ecModel.mat')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06


In [3]:
mae.reactions.ATPM

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x7f995f7b0b10
Stoichiometry,atp_c + h2o_c + 0.160442187835805 prot_standard --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O + 0.160442187835805 prot_standard --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,3.15
Upper bound,1000.0


In [ ]:
mae.medium

In [ ]:
model = cobra.io.read_sbml_model('/home/aditi/Documents/Project/GECKO/userData/myEcoli/models/iJO1366.xml')

In [ ]:
model.medium

In [ ]:
reaction = cobra.Reaction('NGAM')
reaction.name = 'NGAM'
reaction.lower_bound = 3.2 # This is the default
reaction.upper_bound = 3.2  # This is the default

h_c = mae.metabolites.h_c
atp_c = mae.metabolites.atp_c
adp_c = mae.metabolites.adp_c
h2o_c = mae.metabolites.h2o_c
pi_c = mae.metabolites.pi_c

reaction.add_metabolites({
    atp_c: -1.0,
    h2o_c: -1.0,
    adp_c: 1.0,
    h_c: 1.0,
    pi_c: 1.0
})

In [ ]:
mae.add_reactions([reaction])
print(len(mae.reactions))

In [ ]:
def addNGAMreaction(model):
    reaction = cobra.Reaction('NGAM')
    reaction.name = 'NGAM'
    reaction.lower_bound = 1 # This is the default
    reaction.upper_bound = 1  # This is the default

    h_c = model.metabolites.h_c
    atp_c = model.metabolites.atp_c
    adp_c = model.metabolites.adp_c
    h2o_c = model.metabolites.h2o_c
    pi_c = model.metabolites.pi_c

    reaction.add_metabolites({
        atp_c: -1.0,
        h2o_c: -1.0,
        adp_c: 1.0,
        h_c: 1.0,
        pi_c: 1.0
    })
    model.add_reactions([reaction])
    
    print('Successfully added NGAM reaction')
    return model

In [ ]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('/home/aditi/Documents/Project/GECKO/userData/myEcoli/models/ecModel.mat')
    
    mae = addNGAMreaction(mae)
    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iAF1260_core_59p81M.upper_bound = 50
    
    try: rae, model = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: rae, model = np.zeros(dfae_batch.shape[0])
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    #print(rae)
    #rexp = GEMS.aerobic_exp_data()['data']
    
    #print('r2_batch:',r2_score(rexp,rae))
    #print('MSE_ae',MSE(rexp,rae))
    return {'data':np.array(rae)}

In [ ]:
def sharpeschoolhigh_1981(temp, r_tref, e, eh, th, tref):
    tref = 273.15 + tref
    k = 8.62e-05
    boltzmann_term = r_tref * np.exp(e/k * (1/tref - 1/(temp + 273.15)))
    inactivation_term = 1 / (1 + np.exp(eh/k * (1/(th + 273.15) - 1/(temp + 273.15))))
    return boltzmann_term * inactivation_term


In [ ]:
def getNGAMT(T):
    # T is in K, a single value
    def NGAM_function(T):
        return 8.5*(1-(0.62*np.exp((-0.5/(8.617*10**-5))*((1/(273.15+6)-1/(T))))))
        #return 0.740 + 5.893/(1+np.exp(31.920-(T-273.15))) + 6.12e-6*(T-273.15-16.72)**4

    NGAM_T = NGAM_function(T)
    if NGAM_T < NGAM_function(273.15+36):
        return NGAM_function(273.15+36)

    return NGAM_T

In [ ]:
NGAM = []
for T in dfae_batch.index+273.15:
    #print(T)
    NGAM.append(getNGAMT(T))
print(NGAM)

In [ ]:
plt.figure()
plt.plot(dfae_batch.index,NGAM,markersize=5,marker='o')
plt.xlabel('Temperature ($^\circ$C)')
#plt.ylabel('Specific growth rate (h$^{-1}$)')
plt.ylabel('Maintainence')

#plt.legend(fontsize=9,loc='upper left')
plt.tight_layout()
plt.savefig('../results/possibleNGAM.png')
plt.show()

In [ ]:
rae, model = aerobic(params)

In [ ]:
plt.figure()
plt.scatter(dfae_batch.index,dfae_batch['r'].values,label='Exp',color='k')
plt.plot(dfae_batch.index,rae['data'],markersize=5,marker='o',label='Pred')
plt.xlabel('Temperature ($^\circ$C)')
#plt.ylabel('Specific growth rate (h$^{-1}$)')
plt.ylabel('Aerobic $r_{max}$ (h$^{-1}$)')

plt.legend(fontsize=9,loc='upper left')
plt.tight_layout()
#plt.savefig('../results/sim3.pdf')
plt.show()

In [ ]:
mae.summary()

In [ ]:
mae.reactions.usage_prot_P75804

In [ ]:
mae.reactions.usage_prot_P75804.get_coefficient('prot_P75804')

In [ ]:
met = mae.metabolites.prot_pool
T = 273
Tadj=0
for rxn in met.reactions:
    # this is to ignore reaction 'prot_pool_exchange': --> prot_pool
    if len(rxn.metabolites)<2: continue

    uniprot_id = rxn.id.split('_')[-1]
    print(uniprot_id)
    print(rxn.metabolites[met])
    cols = ['dHTH', 'dSTS','dCpu','Topt']
    [dHTH, dSTS,dCpu,topt]=df.loc[uniprot_id,cols]
    fNT = get_fNT(T+Tadj,dHTH,dSTS,dCpu)

    if fNT < 1e-32: fNT = 1e-32
    new_coeff = rxn.metabolites[met]/fNT
    print(fNT)
    #etc.change_rxn_coeff(rxn,met,new_coeff)

In [ ]:
def get_fNT(T,dHTH,dSTS,dCpu):
    '''
    # Calculate the fraction of enzyme in native state
    # dHTH, dSTS are enthalpy and entropy at TH and TS, repspectively
    # dCpu is the heat capacity change unpoin unfolding
    # T, temeperature, a single value or a numpy array, in K
    '''
    R = 8.314;
    dGu = etc.get_dGu(T,dHTH,dSTS,dCpu);
    f = 1/(1+np.exp(-dGu/R/T));
    return f

In [ ]:
mae.reactions.usage_prot_P00935.bounds

In [ ]:
mae = cobra.io.load_matlab_model('../models/ecModel.mat')